# Convert between superoperator representations

In [ ]:
import numpy as np
from forest_benchmarking.superop_conversion import vec, unvec

In [ ]:
I = np.asarray([[1, 0], [0, 1]])
X = np.asarray([[0, 1], [1, 0]])
Y = np.asarray([[0, -1.0j], [1.0j, 0]])
Z = np.asarray([[1, 0], [0, -1]])
H = np.asarray([[1, 1], [1, -1]]) / np.sqrt(2)
CNOT = np.array([[1,0,0,0],[0,1,0,0],[0,0,0,1],[0,0,1,0]])
one_state = np.asarray([[0,0],[0,1]])
zero_state = np.asarray([[1,0],[0,0]])

# vec and unvec 

In [ ]:
A = np.asarray([[1, 2], [3, 4]])

In [ ]:
print(A)
print(" ")
print(vec(A))
print(" ")
print(unvec(vec(A))==A)

# Apply Kraus operators to the excited state

In [ ]:
one_state = np.asarray([[0,0],[0,1]])
one = vec(one_state)
Ad0 = np.asarray([[1, 0], [0, np.sqrt(1-0.1)]])
Ad1 = np.asarray([[0, np.sqrt(0.1)], [0, 0]])
ko = [Ad0, Ad1]
print(ko)
# check kraus ops sum to ID
np.matmul(Ad0.transpose(), Ad0) + np.matmul(Ad1.transpose(), Ad1)

Use Kraus operators to find out put of channel i.e.

$\rho_{out} = A_0 \rho A_0^\dagger + A_1 \rho A_1^\dagger$

Except we only transpose as these matrices have real entries

In [ ]:
rho_out = np.matmul(np.matmul(Ad0, one_state), Ad0.transpose()) \
    + np.matmul(np.matmul(Ad1, one_state), Ad1.transpose())
print(rho_out)

## Kraus to superoperator

Do the case of one Kraus operator i.e. a unitary.

In [ ]:
from forest_benchmarking.superop_conversion import kraus2superop

In [ ]:
zero_state_vec = vec(zero_state)
print(zero_state)
print("")
print(zero_state_vec)

In [ ]:
# Hadamard * |0>
unvec(np.matmul(kraus2superop([H]), zero_state_vec))

Do the case of amplitude damping i.e. two Kraus operators.

In [ ]:
# super op makes sense
one_state_vec = vec(one_state) 
print("vec'ed version \n", np.matmul(kraus2superop(ko), one_state_vec))
print(" ")
print("unvec'ed i.e. rho_out \n", unvec(np.matmul(kraus2superop(ko), one_state_vec)))

In [ ]:
print(kraus2superop([X]))

# Kraus to Choi

In [ ]:
from scipy.linalg import expm
from forest_benchmarking.superop_conversion import kraus2choi
from forest_benchmarking.utils import partial_trace

In [ ]:
choi = kraus2choi(ko)
print(choi)

In [ ]:
prody = np.matmul(choi, np.kron(one_state.transpose(),I) )
print(partial_trace(prody, [1], [2,2]))
print("\n which is the same as rho_out")


In [ ]:
choiCNOT = kraus2choi([CNOT])
print(choiCNOT)

In [ ]:
two_q_id = np.kron(I,I)

print("Lets see if the CNOT action is correct")
print(" ")

print("one one state")
input_state = np.kron(one_state, one_state)
prody = np.matmul(choiCNOT, np.kron(two_q_id, input_state.transpose()))
print(partial_trace(prody, [0,1], [2,2,2,2]))

print("one zero state")
input_stated = np.kron(one_state, zero_state)
prody = np.matmul(choiCNOT, np.kron(two_q_id, input_stated.transpose()))
print(partial_trace(prody, [0,1], [2,2,2,2]))

print("zero one state")
input_stated = np.kron(zero_state,one_state)
prody = np.matmul(choiCNOT, np.kron(two_q_id, input_stated.transpose()))
print(partial_trace(prody, [0,1], [2,2,2,2]))

print("zero zero state")
input_stated = np.kron(zero_state, zero_state)
prody = np.matmul(choiCNOT, np.kron(two_q_id, input_stated.transpose()))
print(partial_trace(prody, [0,1], [2,2,2,2]))

print("\n Story checks out.")